<a href="https://colab.research.google.com/github/cerezamo/NLP_brouillon/blob/master/Camembert_clean_automated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CamemBERT classification model 


In [1]:
import spacy 
import numpy as np 
import pandas as pd 
import os 
os.getcwd()
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


### Set up Colab GPU 

In [4]:
# First you should go in 'Edit' -> 'Notebook settings' -> Add device GPU
import tensorflow as tf

# GPU device name.
device_name = tf.test.gpu_device_name()
device_name

''

Let's now tell torch that one GPU is available 

In [5]:
import torch

if torch.cuda.is_available():  
        
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


Let's install the Hugging Face Library transformer package 

In [6]:
! pip install transformers 

     |████████████████████████████████| 542kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 13.1MB/s 
     |████████████████████████████████| 870kB 20.1MB/s 
     |████████████████████████████████| 3.7MB 21.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=f3d4b49fc971e1db5b324ea3a1a563d662d7d5ad49f96ec1b345fb7c97cd510d
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


### Loading our corpus and preprocessing 

In [0]:
import pandas as pd
# Import medium_df_desq in "files"
# Load the dataset into a pandas dataframe.
#df=pd.read_csv('medium_df_deseq.csv',encoding='utf-8')
df=pd.read_csv('medium_df_deseq.csv',encoding='utf-8')

# We replace the labels in a more normalized way : 0=men, 1=women 
df.sexe=df.sexe.replace(1,0)
df.sexe=df.sexe.replace(2,1)

In [0]:
# Make results reproducible 
seed_val = 2020

In [0]:
def unbalanced_preprocess(df,seed_val):

  from sklearn.model_selection import train_test_split

  #Shuffle the data 
  df_unbalanced=df.sample(frac=1).reset_index()

  # Reduce to the variables we are interested in 
  df_unbalanced=df[['Texte','sexe']]

  # Report the number of speeches in the corpus.
  print('Number of text in the unbalanced corpus : {:,}\n'.format(df_unbalanced.shape[0]))
  prop = (len(df_unbalanced[df_unbalanced.sexe==1])/len(df_unbalanced))*100
  print('Proportions of women in the unbalanced corpus : {}\n'.format(prop))

  # We keep one little sample for evaluation 
  model_unbalanced, dev_unbalanced = train_test_split(df_unbalanced, test_size=0.02,random_state=seed_val)

  return model_unbalanced, dev_unbalanced

In [73]:
df_unbalanced = unbalanced_preprocess(df,seed_val)[0]

Number of text in the unbalanced corpus : 5,000

Proportions of women in the unbalanced corpus : 25.0



In [0]:
def balanced_preprocess(df,seed_val):
  from sklearn.model_selection import train_test_split

  # Let's take a balanced sample (model classifying all in men otherwise)
  df_m = df.loc[df['sexe'] == 0]
  df_f = df.loc[df['sexe'] == 1] 
  df_m = df_m[0:len(df_f)]
  df = df_f.append(df_m)

  #Shuffle the data 
  df=df.sample(frac=1).reset_index()

  # Reduce to the variables we are interested in 
  df_balanced=df[['Texte','sexe']]

  # Report the number of speeches in the corpus.
  print('Number of text in this corpus : {:,}\n'.format(df_balanced.shape[0]))
  prop = (len(df_balanced[df_balanced.sexe==1])/len(df_balanced))*100
  print('Proportions of women in the balanced corpus : {}\n'.format(prop))

  # Keep one little sample for evaluation 
  model_balanced, dev_balanced = train_test_split(df_balanced, test_size=0.02,random_state=seed_val)

  return model_balanced, dev_balanced



In [77]:
df_balanced = balanced_preprocess(df,seed_val)[0]

Number of text in this corpus : 2,500

Proportions of women in the balanced corpus : 50.0



In [0]:
def balanced_splitted(df):
  from sklearn.model_selection import train_test_split

  from itertools import repeat
  n=2500
  chunks, label_split=[],[]
  j=0
  for text in df.Texte :
      txt=[text[i:i+n] for i in range(0, len(text), n)]
      chunks.append(txt)
      label_split.extend(repeat(df.sexe[j], len(txt)))
      j+=1

  chunks = [item for sublist in chunks for item in sublist]
  df=pd.DataFrame([chunks,label_split]).transpose()
  df.columns=['Texte','sexe']
  len(df)

  # Let's take a balanced sample (model classifying all in men otherwise)
  df_m = df.loc[df['sexe'] == 0]
  df_f = df.loc[df['sexe'] == 1] 
  df_m = df_m[0:len(df_f)]
  df = df_f.append(df_m)

  #Shuffle the data 
  df=df.sample(frac=1).reset_index()

  # Reduce to the variables we are interested in 
  df=df[['Texte','sexe']]

  # Put as integer 
  df['sexe'] = df['sexe'].astype(int)

  df_balanced_split= df

  # Report the number of speeches in the corpus.
  print('Number of text in this balanced splitted corpus : {:,}\n'.format(df_balanced_split.shape[0]))
  prop = (len(df_balanced_split[df_balanced_split.sexe==1])/len(df_balanced_split))*100
  print('Proportions of women in the balanced splitted corpus : {}\n'.format(prop))

  # Keep one little sample for evaluation 
  model_balanced_split, dev_balance_split = train_test_split(df_balanced_split, test_size=0.02,random_state=seed_val)
  
  return model_balanced_split, dev_balance_split

In [82]:
df_balanced_split = balanced_splitted(df)[0]

Number of text in this balanced splitted corpus : 13,000

Proportions of women in the balanced splitted corpus : 50.0



**We propose 3 samples to train our model :**


1.   **Unbalanced sample**

We take the raw data without any further treatment.

2.   **Balanced sample**

The second option consist in deleting randomly part of male speeches in order to get a balanced sample. Indeed, in the case of unbalanced sample our model could decide to classify all speakers in the male category which would lead to a 0.75 accuracy in our case study. In order to avoid this we feed the model with the same proportions of male and female speakers. Other kind of treatments exist to deal with unbalanced sample (oversampling for example). This one is the simpliest one and we could argue that there is a possibility that the deleted sample contains important information that we therefore miss. However we believe that in our case this is not a big issue. Our unbalanced sample is quite large for both female and male.

3. **Balanced and splitted sample**

The third option is a response to the max length constraint of BERT models. Our text samples are big and contain much more tokens than the 512 limit. In the first two options we decide to just feed the model with the 512 first tokens and thus delete the rest of them. In this third option we cut the text into x parts containing 500 tokens each. All parts of the speech will serve to feed the model. By this technique we do not loose potential important informations at the end of the text. A lot of other techniques have been employed (see ref !!! PUT). We decide to stick to this one in this project. 



#### Tokenization of our text and preparing to feed CamemBERT

#### Loading the Camembert Tokenizer

In [84]:
# Import Camembert tokenizer
from transformers import CamembertTokenizer
# We choose a right padding side for the moment and we will test for a left padding side on a second stage
tokenizer = CamembertTokenizer.from_pretrained('camembert-base', do_lower_case=False,padding_side='right') #left

In [85]:
# Print the original text.
print(' Original: ', df.Texte[0])

# Print the text split into tokens.
print('Tokenized: ', tokenizer.tokenize(df.Texte[0]))

# Print the text mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(df.Texte[0])))

 Original:  Messieurs,Je suis heureux de vous saluer. Quand je dis que je suis heureux de vous saluer, ce n'est pas une simple affirmation de politesse. Je le disais à l'instant à Monsieur le Ministre de la Défense, M. Richard, c'est pour moi un instant où il y a un peu d'émotion; je vais vous dire pourquoi.Vous êtes la première classe d'âge qui ne fera pas de service militaire. C'est une décision que j'ai prise, il y a deux ans, après une vraie réflexion et un vrai débat. Après tout, le service militaire c'est une vieille tradition nationale, il était plus que centenaire. Il y avait toutes sortes de raisons à cela, notamment la nécessité d'avoir une armée nombreuse et donc d'avoir des jeunes formés aux combats, à l'utilisation des armes de l'époque.On pouvait s'interroger sur la nécessité de poursuivre dans cette voie. Il y avait naturellement des critiques, il y avait beaucoup de jeunes qui se disaient qu'ils perdaient un peu leur temps, d'autres qui étaient satisfaits. Mais il y ava

#### Adding special tokens to the start and end of the text


Preprocessing steps : 


1.   **Add special tokens [CLS] [SEP]** 

According to the documentation we need to add special tokens to the start and end of the text Moreover, for camembert we should add a space between CLS and the first token (not sure here, we have to ask benjamin). 

2.   **Pad and truncate all texts to a single number**

Pretrained transformes like Camembert only accept input of the same length. Our corpus contains large texts and we have to pad them in order to be able to feed Camembert. We will set the max length to a large number in order to get all information possible in the text. We choose a max length of 500 which is almost the maximum (512) "sentence" length  accepted. We are aware that this choice will impact a lot training speed.

3.   **Construct an attention mask**

Attention masks are just set to 1 when the token have to be analyzed and 0 otherwise (padded tokens). All our attention mask should be 1 with this corpus. 



For sake of simplicity and to avoid errors we will use the function encode_plus of the library which is really convenient. 



#### Length and attention mask 

In [0]:
def prepare_to_feed(df,length):
  from torch.utils.data import TensorDataset, random_split
  from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

  texts = df.Texte.values
  labels = df.sexe.values

  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []
  num_truncated_tokens =[]
  # Apply function to our corpus
  for text in texts:
      encoded_dict = tokenizer.encode_plus(
                          text,                      # text
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = length,           # We choose for now a max length of 500.
                          pad_to_max_length = True,    # Pad text to max (marche pas en pad left ?)
                          return_attention_mask = True,   # Construct attention masks
                          return_tensors = 'pt',     # Return pytorch tensors.
                          return_overflowing_tokens =True, # return overflowing token information
                    )
      
      # Map tokens to their id in the dictionnary 
      # We add this to our list    
      input_ids.append(encoded_dict['input_ids'])
  
      #num_truncated_tokens.append(encoded_dict['num_truncated_tokens'])
      
      # 3. Attention masks
      attention_masks.append(encoded_dict['attention_mask'])

  # We convert all this into tensors in order to be able to make it work on GPU 
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)

  # Original text and transformed tensor print 
  print("Let's check for the first text indexes, attention masks and labels")
  print(" ")
  print('Original: ', texts[0])
  print('IDs:', input_ids[0])
  print('Attention masks:', attention_masks[0])
  print('labels',labels[0])


  # Combine all above
  dataset = TensorDataset(input_ids, attention_masks, labels)

  # Let's create a 80-20 train / validation dataset 
  train_size = int(0.8 * len(dataset))
  val_size = len(dataset) - train_size

  train_set, val_set = random_split(dataset, [train_size, val_size])

  print('We have {} training samples'.format(train_size))
  print('We have {} validation samples'.format(val_size))

  # We set the size of the batch lower than what is usually set (16 of 32)
  batch_size = 16

  # We create data loaders for the train and validation dataset. 
  train_dataloader = DataLoader(
              train_set,  # The training samples.
              sampler = RandomSampler(train_set), # Select batches randomly
              batch_size = batch_size # Trains with this batch size.
          )

  val_dataloader = DataLoader(
              val_set, # The validation samples.
              sampler = SequentialSampler(val_set), # Pull out batches sequentially.
              batch_size = batch_size # Evaluate with this batch size.
          )
  
  print('Data loaders created for train [0] and val [1]')

  return train_dataloader, val_dataloader 

In [0]:
prepare_to_feed(df_balanced,length=500)

5 and 6 seem to be the [CLS] and [SEP] special tokens 
